In [ ]:
import sys
import os

project_root = os.path.abspath("../..")

if project_root not in sys.path:
    sys.path.append(project_root)

Project root added: c:\Users\Echelon\Desktop\re\sa-news


In [ ]:
from datapipeline.utils.spark_session import get_spark_session

spark = get_spark_session("Bronze_NewsAPI")

In [ ]:
from pyspark.sql.functions import xxhash64, col

newapi_data_path = "../../sanewsstorage/main/newsapi"

newsapi_df = (
    spark.read
         .option("recursiveFileLookup", "true")
         .parquet(newapi_data_path)
)

newsapi_df.printSchema()

newsapi_df = newsapi_df.withColumn(
    "hash",
    xxhash64(*[col(f"`{c}`") for c in newsapi_df.columns])
)

newsapi_df.show(5)

root
 |-- author: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- url: string (nullable = true)
 |-- urlToImage: string (nullable = true)
 |-- publishedAt: string (nullable = true)
 |-- content: string (nullable = true)
 |-- source.id: string (nullable = true)
 |-- source.name: string (nullable = true)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+--------------------+
|              author|               title|         description|                 url|          urlToImage|         publishedAt|             content|      source.id|         source.name|                hash|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+--------------------+
|          N

In [20]:
from delta.tables import DeltaTable
import os

bronze_path = "../../sanewsstorage/bronze/deltatables/newsapi_delta"

if DeltaTable.isDeltaTable(spark, bronze_path):

    delta_table = DeltaTable.forPath(spark, bronze_path)

    (
        delta_table.alias("t")
        .merge(
            newsapi_df.alias("s"),
            "t.hash = s.hash"
        )
        .whenNotMatchedInsertAll()
        .execute()
    )

else:
    (
        newsapi_df.write
          .format("delta")
          .mode("overwrite")
          .save(bronze_path)
    )

In [21]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(spark, bronze_path)
df = delta_table.toDF()
df.show(5)

files = delta_table.toDF().inputFiles
print(files)


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+-------------------+--------------------+
|              author|               title|         description|                 url|          urlToImage|         publishedAt|             content|         source.id|        source.name|                hash|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+-------------------+--------------------+
|Eugene Boisvert, ...|Rain brings relie...|High rainfall in ...|https://www.abc.n...|https://live-prod...|2026-02-09T02:59:43Z|High rainfall in ...|       abc-news-au|      ABC News (AU)| 2599576790646987199|
|       TOI Education|RNSB Jr Executive...|Rajkot Nagarik Sa...|https://timesofin...|https://static.to...|2026-02-09T02:59:21Z|<ul><li>News</li>...|the-times-of-ind